In [ ]:
#  Copyright 2025 Google LLC. This software is provided as-is, without warranty
#  or representation for any use or purpose. Your use of it is subject to your
#  agreement with Google.

# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.


### Import libraries

In [2]:
import json
import pprint
import requests
from functools import partial
import vertexai.agent_engines

# Getting better output prints 
pretty_print = partial(pprint.pprint, width=140)

See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 

In [ ]:
from data_explorer_agent.agent import root_agent
from data_explorer_agent.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [15]:
# Query agent with streaming query result

for event in agent_engine.stream_query(message="hi!", user_id="test"):
    pretty_print(event)

{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'data_store_context': '\n'
                                                   '**Documents Information Available:**\n'
                                                   '* "THE MAGIC OF NAMES - THEIR ORIGIN AND MEANING". This document is a source for '
                                                   'information regarding the etymology, historical significance, \n'
                                                   'and cultural meanings of various names.\n'
                                                   '* Indexed website, specifically "https://www.names.org/". This website serves as a '
                                                   'broad online resource for details about names, such as their \n'
                                                   'meanings, origins, historical and current popularity trends, and often includes lists '
                                

In [16]:
pretty_print(event['content']['parts'][0]['text'])

('Hello! How can I help you with data analysis or information retrieval today? I can access information about names from a SQL database '
 'containing US names from 1910 to the present, as well as documents about name origins and meanings.\n')


In [17]:
# Running a folow up question to the greting message

# First lets check what are the active sessions for my user...
sessions = agent_engine.list_sessions(user_id='test')
sessions.sessions

[Session(id='e921e0b5-56e6-4dd7-8dfa-27d87a3c8170', app_name='default-app-name', user_id='test', state={}, events=[], last_update_time=1746727043.274502)]

In [18]:
latest_session =  sessions.sessions[-1].id #Getting the lastest session

for event in agent_engine.stream_query(message="What is the meaning of the most popular male name in 1994?", user_id="test", session_id=latest_session):
    pretty_print(event, width=140)

{'actions': {'artifact_delta': {}, 'requested_auth_configs': {}, 'state_delta': {}},
 'author': 'root_agent',
 'content': {'parts': [{'text': 'I need to find the most popular male name in 1994 first. After that, I will use the document data store '
                                'to find the meaning of the name.\n'
                                '\n'},
                       {'function_call': {'args': {'question': 'What was the most popular male name in the USA in 1994?'},
                                          'id': 'adk-8a63703f-4640-4b82-bbf3-2b62e0b70041',
                                          'name': 'call_db_agent'}}],
             'role': 'model'},
 'id': 'lEtEvy2D',
 'invocation_id': 'e-9f245920-8913-49ef-afa0-2f55c9a9aa44',
 'long_running_tool_ids': set(),
 'timestamp': 1746727063.059891}



 sql: Here's how to construct the SQL query to answer the question:

1.  **Determine the Table:** The question asks about names in the USA, and the table `bigquery-public-data.usa_names.usa_1910_2013` and `bigquery-public-data.usa_names.usa_1910_current` seems relevant. Since the question asks about the year 1994, and both tables contain data for that year, we can use either table. For simplicity, I will use `bigquery-public-data.usa_names.usa_1910_2013`.
2.  **Filter for the Year and Gender:** We need to filter the data for the year 1994 and the male gender ('M').
3.  **Aggregate to Find Popularity:** We need to group by name and sum the number of occurrences to find the total number of occurrences for each name.
4.  **Order and Limit:** We need to order the results by the total number of occurrences in descending order and limit the result to the top 1 to find the most popular name.


SELECT
    `name`
  FROM
    `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE year = 1994
   


 run_bigquery_validation final_result: 
 {'query_result': [{'name': 'Michael'}], 'error_message': None}
{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'query_result': [{'name': 'Michael'}],
                             'sql_explorer_agent_output': '```json\n'
                                                          '{\n'
                                                          '  "explain": "The question asks for the most popular male name in the USA in '
                                                          '1994. I first generated an SQL query to retrieve the most popular male name '
                                                          'from the `bigquery-public-data.usa_names.usa_1910_2013` table, filtering by '
                                                          "year 1994 and gender 'M', grouping by name, ordering by the sum of the number "
                                                          'of oc

I0000 00:00:1746727088.476572 6795252 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


{'actions': {'artifact_delta': {}, 'requested_auth_configs': {}, 'state_delta': {}},
 'author': 'root_agent',
 'content': {'parts': [{'text': "The most popular male name in the USA in 1994 was Michael. Now, let's find out the meaning of the name "
                                'Michael.\n'
                                '\n'},
                       {'function_call': {'args': {'question': 'meaning of the name Michael'},
                                          'id': 'adk-9cd5a3b5-f6c1-49e0-be23-0782e2d3b79e',
                                          'name': 'call_document_explorer_agent'}}],
             'role': 'model'},
 'id': 'KfcC4419',
 'invocation_id': 'e-9f245920-8913-49ef-afa0-2f55c9a9aa44',
 'long_running_tool_ids': set(),
 'timestamp': 1746727085.887081}
{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'documents_explorer_agent_output': 'The meaning of the name Michael is "Who is like God?".\n'
                   

In [19]:
for event in agent_engine.stream_query(message="What about the female?", user_id="test", session_id=latest_session):
    pretty_print(event, width=140)

I0000 00:00:1746727097.713501 6796853 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


{'actions': {'artifact_delta': {}, 'requested_auth_configs': {}, 'state_delta': {}},
 'author': 'root_agent',
 'content': {'parts': [{'text': 'I need to find the most popular female name in 1994 first. After that, I will use the document data store '
                                'to find the meaning of the name.\n'
                                '\n'},
                       {'function_call': {'args': {'question': 'What was the most popular female name in the USA in 1994?'},
                                          'id': 'adk-0f185077-4dc0-486c-bcb4-d66e7acd68b4',
                                          'name': 'call_db_agent'}}],
             'role': 'model'},
 'id': '4vLLLJ9N',
 'invocation_id': 'e-64c4472e-b5d1-4ef9-890e-3bd3a4cbdb42',
 'long_running_tool_ids': set(),
 'timestamp': 1746727095.208529}



 sql: Here's how to construct the SQL query to answer the question:

1.  **Identify the table:** The question is about names in the USA, and the table `bigquery-public-data.usa_names.usa_1910_2013` and `bigquery-public-data.usa_names.usa_1910_current` seems relevant. Since the question asks about the year 1994, and both tables contain data for that year, we should use both tables and combine the results.
2.  **Filter by year and gender:** We need to filter the data for the year 1994 and for female names (gender = 'F').
3.  **Aggregate to find the most popular name:** We need to group by name and sum the number of occurrences to find the total number of occurrences for each name.
4.  **Order and limit to find the most popular:** We need to order the results by the total number of occurrences in descending order and limit the result to 1 to find the most popular name.
5.  **Combine the two tables:** Use a `UNION ALL` to combine the results from the two tables.


SELECT
    name
  FROM
 


 run_bigquery_validation final_result: 
 {'query_result': [{'name': 'Jessica'}], 'error_message': None}
{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'query_result': [{'name': 'Jessica'}],
                             'sql_explorer_agent_output': '```json\n'
                                                          '{\n'
                                                          '"explain": "I have validated the SQL query using the run_bigquery_validation '
                                                          'tool and it is valid. The result shows that the most popular female name in the '
                                                          'USA in 1994 was Jessica.",\n'
                                                          '"sql": "SELECT\\n    name\\n  FROM\\n    (\\n      SELECT\\n          '
                                                          '`usa_1910_2013`.name AS name,\\n          SUM(`usa_1910_2

I0000 00:00:1746727117.917588 6796853 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


{'actions': {'artifact_delta': {}, 'requested_auth_configs': {}, 'state_delta': {}},
 'author': 'root_agent',
 'content': {'parts': [{'text': "The most popular female name in the USA in 1994 was Jessica. Now, let's find out the meaning of the name "
                                'Jessica.\n'
                                '\n'},
                       {'function_call': {'args': {'question': 'meaning of the name Jessica'},
                                          'id': 'adk-65d18cc6-a5d4-482a-8175-12336a35fae7',
                                          'name': 'call_document_explorer_agent'}}],
             'role': 'model'},
 'id': '6qXmTtgk',
 'invocation_id': 'e-64c4472e-b5d1-4ef9-890e-3bd3a4cbdb42',
 'long_running_tool_ids': set(),
 'timestamp': 1746727115.484625}
{'actions': {'artifact_delta': {},
             'requested_auth_configs': {},
             'state_delta': {'documents_explorer_agent_output': 'I could not find information about the meaning of the name Jessica in the '

In [24]:
# Registering Feedback for last inraction 
last_invocation_id = event['invocation_id']


feed_back = {
    'score': 5,
    'text': "It was a great response!",
    'invocation_id': last_invocation_id,
    'user_id': 'test'
}


agent_engine.register_feedback(feed_back)

### Remote Testing

In [ ]:
# Replace with your Agent Engine ID
AGENT_ENGINE_ID = "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"

In [ ]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [ ]:
for event in remote_agent_engine.stream_query(message="hi!", user_id="test"):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

## If you are using Cloud Run

### Local Testing

> You can run the application locally via the `make backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_id = "test_session_456"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions/{session_id}"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")

#### Send a message
Send a message to the backend service and receive a streaming response


In [4]:
message_data = {
    "app_name": "data_explorer_agent",
    "user_id": 'user',
    "session_id": 'eca7976b-ee70-4dbc-9e48-a76f14a2b2e2',
    "new_message": {"role": "user", "parts": [{"text": "What is the meaning of this name?"}]},
    "streaming": True,
}
headers = {"Content-Type": "application/json"}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

Message send status code: 200
Received event: {'content': {'parts': [{'text': 'I'}], 'role': 'model'}, 'partial': True, 'usageMetadata': {'trafficType': 'ON_DEMAND'}, 'invocationId': 'e-2e3d59c3-5325-4f16-a0a2-91e5813e65de', 'author': 'root_agent', 'actions': {'stateDelta': {}, 'artifactDelta': {}, 'requestedAuthConfigs': {}}, 'id': 'ja9RJSgE', 'timestamp': 1748389275.373852}
Received event: {'content': {'parts': [{'text': ' can query the document data store to find the meaning of the name "Michael".\n\n'}], 'role': 'model'}, 'partial': True, 'usageMetadata': {'trafficType': 'ON_DEMAND'}, 'invocationId': 'e-2e3d59c3-5325-4f16-a0a2-91e5813e65de', 'author': 'root_agent', 'actions': {'stateDelta': {}, 'artifactDelta': {}, 'requestedAuthConfigs': {}}, 'id': 'auzxVJya', 'timestamp': 1748389275.373852}
Received event: {'content': {'parts': [{'text': 'I can query the document data store to find the meaning of the name "Michael".\n\n'}], 'role': 'model'}, 'usageMetadata': {'candidatesTokenCoun

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_id = "test_session_456"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions/{session_id}"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")